# Consumption Layer

In this notebook, we make the reference tables that will be consumed. These tables will hold clean reference data, as well as a change history of this data. For each table from the staging layer, we add three new fields to the change history:
*   `effective_time<TIMESTAMP>` to indicate the time that the record became active in the table.
*   `discontinue_time<TIMESTAMP>` to indicate the time that the record became inactive in the table.
*   `active_flag<BOOL>` whether the record is active or not.

Note that we don't technically need `active_flag`. We can infer it from the `discontinue_time` value. However, it is convenient to have this field for querying and reporting.


In [ ]:
%%bigquery
select table_name from airline_stg.INFORMATION_SCHEMA.TABLES order by table_name

Query is running:   0%|          |

Downloading:   0%|          |

,table_name
0,Air_Carrier
1,Airport
2,Flight
3,Flight_History
4,Meal
5,Meal_Service
6,Snack
7,Snack_Service


# Air_Carrier

In [6]:
%%bigquery
create or replace table airline_csp.Air_Carrier(
  airline_id	INT64 not null,
  airline_name	STRING not null,
  airline_code	STRING,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (airline_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Air_Carrier;

Query is running:   0%|          |

""


In [8]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Air_Carrier'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,airline_id,INT64,NO,NULL,Air_Carrier.pk$
1,airline_name,STRING,NO,NULL,None
2,airline_code,STRING,YES,NULL,None
3,data_source,STRING,NO,NULL,None
4,load_time,TIMESTAMP,NO,NULL,None
5,effective_time,TIMESTAMP,NO,current_timestamp(),Air_Carrier.pk$
6,discontinue_time,TIMESTAMP,YES,NULL,None
7,status_flag,BOOL,NO,NULL,None


In [9]:
%%bigquery
select * from airline_csp.Air_Carrier
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,airline_name,airline_code,data_source,load_time,effective_time,discontinue_time,status_flag
0,19185,Air Pacific Inc.,EKA,bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,NaT,True
1,19245,Maxair Inc.,MAX,bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,NaT,True
2,19579,Heavylift Cargo Airlines Lt,HLQ,bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,NaT,True
3,19961,Air Transat,TS,bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,NaT,True
4,21569,Lauda Motion GmbH,LDM,bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,NaT,True


# Airport

In [11]:
%%bigquery
create or replace table airline_csp.Airport(
  iata STRING not null,
  icao STRING,
  name STRING,
  city STRING not null,
  state STRING,
  country STRING not null,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (iata, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Airport;

Query is running:   0%|          |

""


In [12]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Airport'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,iata,STRING,NO,NULL,Airport.pk$
1,icao,STRING,YES,NULL,None
2,name,STRING,YES,NULL,None
3,city,STRING,NO,NULL,None
4,state,STRING,YES,NULL,None
5,country,STRING,NO,NULL,None
6,data_source,STRING,NO,NULL,None
7,load_time,TIMESTAMP,NO,NULL,None
8,effective_time,TIMESTAMP,NO,current_timestamp(),Airport.pk$
9,discontinue_time,TIMESTAMP,YES,NULL,None


In [13]:
%%bigquery
select * from airline_csp.Airport
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,iata,icao,name,city,state,country,data_source,load_time,effective_time,discontinue_time,status_flag
0,NDJ,None,Hassan Djamous,Ndjamena,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-17 16:38:34.826089+00:00,NaT,True
1,FYT,None,Faya-Largeau Airport,Faya-Largeau,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-17 16:38:34.826089+00:00,NaT,True
2,SRH,None,Sarh Airport,Sarh,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-17 16:38:34.826089+00:00,NaT,True
3,MQQ,None,Moundou Airport,Moundou,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-17 16:38:34.826089+00:00,NaT,True
4,AMO,None,Mao Airport,Mao,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-17 16:38:34.826089+00:00,NaT,True


# Flight

In [16]:
%%bigquery
create or replace table airline_csp.Flight(
  fl_num STRING not null,
  op_carrier_airline_id INT64 not null,
  op_carrier_fl_num INT64 not null,
  origin_airport STRING not null,
  dest_airport STRING not null,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (fl_num, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Flight;

Query is running:   0%|          |

""


In [18]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Flight'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,fl_num,STRING,NO,NULL,Flight.pk$
1,op_carrier_airline_id,INT64,NO,NULL,None
2,op_carrier_fl_num,INT64,NO,NULL,None
3,origin_airport,STRING,NO,NULL,None
4,dest_airport,STRING,NO,NULL,None
5,data_source,STRING,NO,NULL,None
6,load_time,TIMESTAMP,NO,NULL,None
7,effective_time,TIMESTAMP,NO,current_timestamp(),Flight.pk$
8,discontinue_time,TIMESTAMP,YES,NULL,None
9,status_flag,BOOL,NO,NULL,None


In [19]:
%%bigquery
select * from airline_csp.Flight
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,op_carrier_airline_id,op_carrier_fl_num,origin_airport,dest_airport,data_source,load_time,effective_time,discontinue_time,status_flag
0,19790667ABEATL,19790,667,ABE,ATL,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:12:02.623941+00:00,NaT,True
1,197901367ABEATL,19790,1367,ABE,ATL,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:12:02.623941+00:00,NaT,True
2,203665569ABEATL,20366,5569,ABE,ATL,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:12:02.623941+00:00,NaT,True
3,203665573ABEATL,20366,5573,ABE,ATL,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:12:02.623941+00:00,NaT,True
4,203665575ABEATL,20366,5575,ABE,ATL,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:12:02.623941+00:00,NaT,True


# Flight_History

In [23]:

%%bigquery
create or replace table airline_csp.Flight_History(
  fl_date DATE not null,
  fl_num STRING not null,
  tail_num STRING,
  crs_dep_time INT64 not null,
  dep_time INT64,
  dep_delay INT64,
  dep_delay_new INT64,
  arr_time INT64,
  arr_delay INT64,
  arr_delay_new INT64,
  cancelled INT64,
  cancellation_code STRING,
  crs_elapsed_time INT64,
  actual_elapsed_time INT64,
  carrier_delay INT64,
  weather_delay INT64,
  nas_delay INT64,
  security_delay INT64,
  late_aircraft_delay INT64,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (fl_date, fl_num, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Flight_History;

Query is running:   0%|          |

""


In [25]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Flight_History'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,fl_date,DATE,NO,NULL,Flight_History.pk$
1,fl_num,STRING,NO,NULL,Flight_History.pk$
2,tail_num,STRING,YES,NULL,None
3,crs_dep_time,INT64,NO,NULL,None
4,dep_time,INT64,YES,NULL,None
5,dep_delay,INT64,YES,NULL,None
6,dep_delay_new,INT64,YES,NULL,None
7,arr_time,INT64,YES,NULL,None
8,arr_delay,INT64,YES,NULL,None
9,arr_delay_new,INT64,YES,NULL,None


In [24]:
%%bigquery
select * from airline_csp.Flight_History
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_date,fl_num,tail_num,crs_dep_time,dep_time,dep_delay,dep_delay_new,arr_time,arr_delay,arr_delay_new,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,data_source,load_time,effective_time,discontinue_time,status_flag
0,2018-08-04,19805103DFWBWI,N972TW,705,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:57:08.983592+00:00,NaT,True
1,2018-08-28,193932032MDWCHS,N554WN,2155,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:57:08.983592+00:00,NaT,True
2,2018-08-04,204524440MYRDCA,N102HQ,1327,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:57:08.983592+00:00,NaT,True
3,2018-08-24,20436866MSODEN,None,1913,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:57:08.983592+00:00,NaT,True
4,2018-08-11,204092579BOSEWR,N273JB,1506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-17 17:57:08.983592+00:00,NaT,True


# Meal_Service

In [26]:
%%bigquery
create or replace table airline_csp.Meal_Service(
  fl_num STRING not null,
  meal_id	INT64 not null,
  airline_id INT64 not null,
  service_category STRING,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (fl_num, meal_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Meal_Service;

Query is running:   0%|          |

""


In [27]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Meal_Service'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,fl_num,STRING,NO,NULL,Meal_Service.pk$
1,meal_id,INT64,NO,NULL,Meal_Service.pk$
2,airline_id,INT64,NO,NULL,None
3,service_category,STRING,YES,NULL,None
4,data_source,STRING,NO,NULL,None
5,load_time,TIMESTAMP,NO,NULL,None
6,effective_time,TIMESTAMP,NO,current_timestamp(),Meal_Service.pk$
7,discontinue_time,TIMESTAMP,YES,NULL,None
8,status_flag,BOOL,NO,NULL,None


In [28]:
%%bigquery
select * from airline_csp.Meal_Service
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,meal_id,airline_id,service_category,data_source,load_time,effective_time,discontinue_time,status_flag
0,19930813LAXOGG,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-17 18:00:47.996713+00:00,NaT,True
1,1993055SCCBRW,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-17 18:00:47.996713+00:00,NaT,True
2,199301359BOSSFO,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-17 18:00:47.996713+00:00,NaT,True
3,19930422SEALAS,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-17 18:00:47.996713+00:00,NaT,True
4,19930617LASSEA,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-17 18:00:47.996713+00:00,NaT,True


# Snack_Service

In [30]:
%%bigquery
create or replace table airline_csp.Snack_Service(
  fl_num STRING not null,
  snack_id STRING not null,
  airline_id INT64 not null,
  service_category STRING,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (fl_num, snack_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Snack_Service;

Query is running:   0%|          |

""


In [31]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Meal_Service'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,fl_num,STRING,NO,NULL,Meal_Service.pk$
1,meal_id,INT64,NO,NULL,Meal_Service.pk$
2,airline_id,INT64,NO,NULL,None
3,service_category,STRING,YES,NULL,None
4,data_source,STRING,NO,NULL,None
5,load_time,TIMESTAMP,NO,NULL,None
6,effective_time,TIMESTAMP,NO,current_timestamp(),Meal_Service.pk$
7,discontinue_time,TIMESTAMP,YES,NULL,None
8,status_flag,BOOL,NO,NULL,None


In [32]:
%%bigquery
select * from airline_csp.Snack_Service
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,snack_id,airline_id,service_category,data_source,load_time,effective_time,discontinue_time,status_flag
0,203044529SLCASE,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-17 18:02:31.735434+00:00,NaT,True
1,203044119FSMATL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-17 18:02:31.735434+00:00,NaT,True
2,203044562SBNATL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-17 18:02:31.735434+00:00,NaT,True
3,203044518VLDATL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-17 18:02:31.735434+00:00,NaT,True
4,203043609DTWBDL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-17 18:02:31.735434+00:00,NaT,True


# Meal

In [33]:
%%bigquery
create or replace table airline_csp.Meal(
  meal_id	INT64 not null,
  meal_name STRING not null,
  meal_image STRING,
  cat_name STRING,
  tags STRING,
  area STRING,
  ingredient1 STRING,
  ingredient2 STRING,
  ingredient3 STRING,
  ingredient4 STRING,
  ingredient5 STRING,
  source STRING,
  youtube STRING,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (meal_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Meal;

Query is running:   0%|          |

""


In [35]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Meal'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,meal_id,INT64,NO,NULL,Meal.pk$
1,meal_name,STRING,NO,NULL,None
2,meal_image,STRING,YES,NULL,None
3,cat_name,STRING,YES,NULL,None
4,tags,STRING,YES,NULL,None
5,area,STRING,YES,NULL,None
6,ingredient1,STRING,YES,NULL,None
7,ingredient2,STRING,YES,NULL,None
8,ingredient3,STRING,YES,NULL,None
9,ingredient4,STRING,YES,NULL,None


In [36]:
%%bigquery
select * from airline_csp.Meal
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,meal_id,meal_name,meal_image,cat_name,tags,area,ingredient1,ingredient2,ingredient3,ingredient4,ingredient5,source,youtube,data_source,load_time,effective_time,discontinue_time,status_flag
0,52827,Massaman Beef curry,https://www.themealdb.com/images/media/meals/t...,Beef,Curry,Thai,Peanuts,Coconut cream,Massaman curry paste,Beef,Potatoes,https://www.bbcgoodfood.com/recipes/420631/bee...,https://www.youtube.com/watch?v=mVxgZSCU9_g,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-17 18:06:10.148700+00:00,NaT,True
1,52979,Bitterballen (Dutch meatballs),https://www.themealdb.com/images/media/meals/l...,Beef,"DinnerParty,HangoverFood,Alcoholic",Dutch,Butter,Flour,Beef Stock,Onion,Parsley,https://www.holland.com/global/tourism/informa...,https://www.youtube.com/watch?v=q8AKfYUtDuM,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-17 18:06:10.148700+00:00,NaT,True
2,53006,Moussaka,https://www.themealdb.com/images/media/meals/c...,Beef,None,Greek,Beef,Aubergine,Greek Yogurt,Egg,Parmesan,https://www.bbcgoodfood.com/recipes/must-make-...,https://www.youtube.com/watch?v=8U_29i9Qp5U,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-17 18:06:10.148700+00:00,NaT,True
3,52998,Corned Beef and Cabbage,https://www.themealdb.com/images/media/meals/x...,Beef,None,Irish,Beef Brisket,Small Potatoes,Carrots,Cabbage,None,None,None,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-17 18:06:10.148700+00:00,NaT,True
4,52781,Irish stew,https://www.themealdb.com/images/media/meals/s...,Beef,"Stew,Meat",Irish,whole wheat,lamb loin chops,olive oil,shallots,carrots,http://www.ottolenghi.co.uk/recipes/meat/irish...,https://www.youtube.com/watch?v=kYH2qJXnSMo,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-17 18:06:10.148700+00:00,NaT,True


# Snack

In [42]:
%%bigquery
create or replace table airline_csp.Snack(
  snack_id STRING not null,
  url	STRING not null,
  product_name STRING,
  brands STRING,
  categories STRING not null,
  countries_en STRING,
  ingredients_text STRING,
  image_url	STRING,
  created_time TIMESTAMP not null,
  last_modified_time TIMESTAMP not null,
  data_source	STRING not null,
  load_time	TIMESTAMP not null,
  effective_time TIMESTAMP default current_timestamp() not null,
  discontinue_time TIMESTAMP,
  status_flag BOOL not null,
  primary key (snack_id, effective_time) not enforced)
  as select *, current_timestamp(), null, true
  from airline_stg.Snack;

Query is running:   0%|          |

""


In [44]:
%%bigquery
select c.column_name, c.data_type, c.is_nullable, c.column_default, k.constraint_name
from airline_csp.INFORMATION_SCHEMA.COLUMNS c left join airline_csp.INFORMATION_SCHEMA.KEY_COLUMN_USAGE k
on c.table_name = k.table_name and c.column_name = k.column_name
where c.table_name = 'Snack'
order by c.ordinal_position;

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type,is_nullable,column_default,constraint_name
0,snack_id,STRING,NO,NULL,Snack.pk$
1,url,STRING,NO,NULL,None
2,product_name,STRING,YES,NULL,None
3,brands,STRING,YES,NULL,None
4,categories,STRING,NO,NULL,None
5,countries_en,STRING,YES,NULL,None
6,ingredients_text,STRING,YES,NULL,None
7,image_url,STRING,YES,NULL,None
8,created_time,TIMESTAMP,NO,NULL,None
9,last_modified_time,TIMESTAMP,NO,NULL,None


In [43]:
%%bigquery
select * from airline_stg.Snack
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,snack_id,url,product_name,brands,categories,countries_en,ingredients_text,image_url,created_time,last_modified_time,data_source,load_time
0,3760138351616,http://world-en.openfoodfacts.org/product/3760...,les chips!,None,"Aliments et boissons à base de végétaux, Alime...",France,None,https://images.openfoodfacts.org/images/produc...,2018-07-30 09:37:49+00:00,2018-08-07 07:54:32+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00
1,3497917002697,http://world-en.openfoodfacts.org/product/3497...,Chips saveur poivre,None,"Aliments et boissons à base de végétaux, Alime...",France,None,https://images.openfoodfacts.org/images/produc...,2020-06-10 18:21:17+00:00,2021-07-07 09:36:34+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00
2,8437000299554,http://world-en.openfoodfacts.org/product/8437...,Chips Sel Et Poivre,None,"Aliments et boissons à base de végétaux, Alime...",France,None,https://images.openfoodfacts.org/images/produc...,2018-03-04 11:13:30+00:00,2020-02-26 19:45:17+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00
3,5400101242562,http://world-en.openfoodfacts.org/product/5400...,Ribblechips Sel & Poivre 200g,None,"Aliments et boissons à base de végétaux, Alime...","Belgium,France",None,https://images.openfoodfacts.org/images/produc...,2018-05-06 10:10:29+00:00,2021-01-17 15:55:24+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00
4,2960685,http://world-en.openfoodfacts.org/product/0296...,Chips sel et poivre,None,"Aliments et boissons à base de végétaux, Alime...",France,None,https://images.openfoodfacts.org/images/produc...,2018-12-08 14:34:39+00:00,2018-12-12 09:35:20+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00


# Foreign Keys

We have to relax referential integrity in this layer. This is because we have added the `effective_date` to the primary key and cannot create foreign keys due to this additional field. That's why it's very important to verify referential integrity in the staging layer (!)  